# Ne(TAP)work-Analysis

# Introduzione

L'idea di questo progetto è quella di creare una data pipeline in real-time, sniffando i pacchetti della propria rete locale per monitorare le abitudini dell'utente tracciando i siti visitati e facendo una predizione sul consumo dei pacchetti nel futuro prossimo.

# Pipeline
<div>
    <img style="float: left;" src="images/pipeline_png.png" width="50%"> 
    <ul style="float: center; padding-top: 150px; padding-left: 250px;">
        <li>Data source: <span style="color: #1B85D4">Wireshark</span></li>
        <li>Data ingestion: <span style="color: #D74FA9">Logstash</span></li>
        <li>Data streaming: <span style="color: #D8A112">Kafka</span></li>
        <li>Data processing: <span style="color: #60D15A">Spark</span></li>
        <li>Data storage: <span style="color: #D74FA9">Elasticsearch</span></li>
        <li>Data visualization: <span style="color: #D74FA9">Kibana</span></li>
    </ul>
</div>

 # Wireshark (Tshark)
<img style="float: left;" src="images/wireshark.png" width="30%"> 

Wireshark è un packet sniffer utilizzato per l'analisi della rete. 

È simile a tcpdump ma è dotato di un'interfaccia grafica, ed è più avanzato poichè dotato di filtri.

Abbiamo utilizzato la versione da command line (Tshark) per loggare alcuni dati dei pacchetti catturati su file. 

<img style="float: center;" src="images/meme1.jpg" width="30%"> 

# Logstash

<img style="float: left;" src="images/logstash3.png" width="50%"> 

## Centralizza, trasforma e conserva i tuoi dati

Logstash è una pipeline gratuita e aperta per l'elaborazione dei dati lato server che ingerisce i dati da una moltitudine di fonti, li trasforma e poi li invia al tuo "stash" preferito.

L'utilizzo di Logstash per questa pipeline è fonfamentale, perché grazie alla sua potenza, con semplicità riusciamo a raccogliere una grandissima quantità di pacchetti catturati da wireshark per poi incanalarli in molteplici output tra cui Kafka per lo streaming e stdout per avere una sorta di log.

Inoltre con Logstash si ha la possibilità di filtare ed arricchire i dati in ingresso.
Nel nostro caso abbiamo aggiunto il filtro geoip per ottenere latitudine e longitudine di un ip.

In [ ]:
input {
  file {
    path => "/usr/share/logstash/pcap/pcap.csv"
    start_position => "beginning"
  }
}

filter {
  csv {
    source => "message"
    columns => [ "timestamp", "ip_src", "hostname", "port"]
  }
  
  date {
    match => [ "timestamp", "UNIX" ]
  } 
  geoip {
    source => "hostname"
    fields => ["dma_code", "continent_code","country_name","city_name","country_code2","location","ip","region_name"] 
  }
  dns {
    reverse => [ "hostname" ]
    action => "replace"
  }

  mutate {
    convert => [ "port", "integer" ]
    remove_field => ["timestamp", "@version", "message", "host", "path"]
  }
}

output {
  kafka {
    codec => json
    topic_id => "tap"
    bootstrap_servers => "kafkaserver:9092"
  }
  stdout {}
}

<img style="float: left;" src="images/kafka3.png" width="60%"> 

Apache Kafka è una piattaforma open-source di streaming di eventi distribuiti utilizzata da migliaia di aziende per pipeline di dati ad alte prestazioni, analisi in streaming, integrazione dei dati e applicazioni mission-critical.

L'utilizzo di Kafka è fondamentale per mettere in comunicazione Logstash e Spark.

Abbiamo utilizzato anche Kafka UI per accertarci che la pipeline funzionasse.

<img style="float: left;" src="images/meme2.jpg" width="25%"> 

<img style="float: left;" src="images/spark.png" width="60%"> 

## Lightning-fast unified analytics engine

### Speed
Esegui i carichi di lavoro 100 volte più velocemente. Apache Spark raggiunge alte prestazioni sia per i dati in batch che in streaming, utilizzando uno scheduler DAG all'avanguardia, un ottimizzatore di query e un motore di esecuzione fisica.

### Ease of Use
Scrivi velocemente applicazioni in Java, Scala, Python, R e SQL. Spark offre oltre 80 operatori di alto livello che rendono facile costruire applicazioni parallele. E puoi usarlo interattivamente dalle shell di Scala, Python, R e SQL.

### Generality
Combina SQL, streaming e analisi complesse.
### Runs Everywhere

# Ease of use?!?!
<img style="float: left;" src="images/meme3.jpg" width="40%"> 

# Machine Learning in Spark

<img style="float: left;" src="images/meme4.jpg" width="30%"> 

Per la predizione dei pacchetti ci siamo affidati ai DataFrame di pandas e al regressore lineare della libreria Scikit-Learn.

<img style="float: left;" src="images/elastic2.png"> 

Elasticsearch è un motore di ricerca e analisi distribuito e RESTful. Come cuore dell'Elastic Stack, memorizza centralmente una grande quantità di dati pur garantendo una ricerca veloce come un fulmine, una pertinenza regolata con precisione e potenti analisi che si scalano con facilità.

<img style="float: left;" src="images/kibana.png"> 

## La tua finestra sull'Elastic Stack

Kibana è un'interfaccia utente gratuita e aperta che ti permette di visualizzare i tuoi dati Elasticsearch e navigare nell'Elastic Stack. Fai qualsiasi cosa, dal monitoraggio del carico delle query alla comprensione del modo in cui le richieste fluiscono attraverso le tue app.

Nel nostro caso Kibana è fondamentale per la visualizzazione di una mappa dettagliata con le posizioni geografiche dei vari ip e anche per la visualizzazione delle predizioni sul numero dei pacchetti futuri.

# DEMO

Per avviare la demo bisogna:

 - Uppare il docker-compose
 - Avviare lo script run.sh che si trova all'interno della cartella logstash

Link utili:
 
 - http://localhost:5601 (Kibana)
 - http://localhost:8080 (Kafka-UI)
 - http://localhost:4040 (Spark)

# Ringraziamenti

<img style="float: left;" src="images/meme5.jpg" width="40%"> 
<img style="float: right;" src="images/meme6.jpg" width="37%"> 

<center><h1>FINE</h1></center>

###### Credits: Simone Benedetto, Valerio Catania.